In [2]:
import pyodbc
import pandas as pd
from pycaret.classification import setup, compare_models

print("All libraries imported successfully!")

All libraries imported successfully!


In [3]:
import pyodbc
import pandas as pd

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"        # default instance
    "DATABASE=LOAN_PORTFOLIO_DB;"
    "Trusted_Connection=yes"   # use Windows Authentication
)

In [4]:
customer_df = pd.read_sql("SELECT * FROM CUSTOMER_DETAILS", conn)
account_df = pd.read_sql("SELECT * FROM ACCOUNT_DETAILS", conn)
transaction_df = pd.read_sql("SELECT * FROM TRANSACTION_DETAILS", conn)
loan_cashflow_df = pd.read_sql("SELECT * FROM LOAN_CASHFLOW", conn)
repayment_df = pd.read_sql("SELECT * FROM REPAYMENT", conn)

In [5]:
account_df

,MASKED_ID,ACC_MASKED_ID,TERM,TERM_AMOUNT,OOD,ACTIVE_PRODUCT,ORIG_CONTRACT_DATE,ACCT_CLOSE_DATE,ACCT_STATUS,MONTHEND_CONVERTED_BALANCE,CONVERTED_BALANCE,JUN_25,JUL_25,AUG_25,SEP_25,OCT_25,NOV_25
0,000454,BB00106DV0L7,None,NaN,NaN,SAVINGS ACCOUNT,2022-11-24,None,ACTIVE,4.207961e+05,1.761807e+04,5.816745e+04,7.188120e+04,1.290630e+03,1.952680e+03,1.506377e+04,2.226882e+05
1,000454,BB00106F1DX0,60M,500000.0,NaN,FIXED DEPOSIT ACCOUNT,2022-11-24,None,ACTIVE,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05,5.000000e+05
2,000454,BB00106L3ZVN,1097D,400000.0,NaN,LOAN ACCOUNT,2022-11-24,2025-12-03,CLOSE,NaN,NaN,-6.444448e+04,-5.304663e+04,-4.322869e+04,-3.287300e+04,-2.233317e+04,-1.130280e+04
3,000454,BB00133NSWRT,369D,2375000.0,NaN,LOAN ACCOUNT,2022-12-21,2023-11-10,CLOSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000454,BB00138PNPFT,60M,2500000.0,NaN,FIXED DEPOSIT ACCOUNT,2022-12-16,None,ACTIVE,2.500000e+06,2.500000e+06,2.500000e+06,2.500000e+06,2.500000e+06,2.500000e+06,2.500000e+06,2.500000e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,9998721,BB09079RPQ53,None,NaN,NaN,SAVINGS ACCOUNT,2014-06-30,None,ACTIVE,3.654700e+02,9.254700e+02,2.586855e+04,2.701799e+04,3.078505e+04,4.528308e+04,2.068335e+04,8.629330e+03
8516,9999535,BB02118R8W32,None,NaN,NaN,SAVINGS ACCOUNT,2024-11-25,None,ACTIVE,8.833616e+04,8.211600e+02,1.016601e+05,1.156048e+06,5.234788e+04,6.728861e+04,1.164431e+05,1.327792e+05
8517,9999535,BB09079FWV46,None,NaN,NaN,SAVINGS ACCOUNT,2014-07-18,None,ACTIVE,7.789401e+05,2.758661e+05,1.404001e+04,6.464130e+03,1.243665e+05,7.613335e+04,2.626008e+05,8.083205e+05
8518,9999658,BB09079VJS24,None,NaN,NaN,SAVINGS ACCOUNT,2014-04-30,None,ACTIVE,5.454049e+04,4.559549e+04,5.743928e+04,6.547739e+04,7.128710e+04,7.698738e+04,7.129555e+04,4.570675e+04


In [6]:
customer_df
customer_df

,MASKED_ID,NAME_MASKED_ID,AGE,GENDER,CUSTOMER_RISK,CUSTOMER_RISK_NAME,OCCUPATION,DISTRICT,TARGET_CODE,TARGET_DESC,MARITAL_STATUS,DATE_OF_BIRTH,EMPLOYMENT_STATUS,MASKED_LEGAL_ID,LEGAL_DOC_NAME,INDUSTRY_CODE,INDUSTRY
0,000454,CUSTOMER_356,36,FEMALE,1.0,LOW,Entrepreneur / Business Owners,ANURADHAPURA,948,Signature,MARRIED,1989-02-12,UNEMPLOYED,673211788H,NIC,9908088888,HEALTH & SOCIAL-OTHER SOCIAL SERVICES
1,000866,CUSTOMER_410,27,FEMALE,1.0,LOW,Garment & Related Trade Workers,ANURADHAPURA,908,Premier,SINGLE,1998-04-19,EMPLOYED,764988120H,NIC,9848688888,"TRADE-GARMENTS, CULTURAL AND RECREATION GOODS"
2,001135,CUSTOMER_455,41,FEMALE,1.0,LOW,Private Sector Non-Executives,HAMBANTOTA,988,Essential,None,1984-03-22,EMPLOYED,623608269H,NIC,9908088888,HEALTH & SOCIAL-OTHER SOCIAL SERVICES
3,001538,CUSTOMER_899,25,FEMALE,1.0,LOW,Finance Professionals,GALLE,988,Essential,None,2000-03-23,EMPLOYED,088836189142,EIC,9918688888,CONSUMPTION-ANY OTHER CONSUMPTION RELATED LOAN...
4,001906,CUSTOMER_2307,61,FEMALE,1.0,LOW,Entrepreneur / Business Owners,COLOMBO,948,Signature,MARRIED,1964-09-07,UNEMPLOYED,425399959W,NIC,9918688888,CONSUMPTION-ANY OTHER CONSUMPTION RELATED LOAN...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,9998375,CUSTOMER_969,39,FEMALE,1.0,LOW,Private Sector Executives,BADULLA,988,Essential,SINGLE,1986-06-08,EMPLOYED,644488129H,NIC,9918688888,CONSUMPTION-ANY OTHER CONSUMPTION RELATED LOAN...
2505,9998721,CUSTOMER_833,34,MALE,2.0,MEDIUM,Entrepreneur / Business Owners,COLOMBO,988,Essential,SINGLE,1991-04-21,BUSINESS,799900186H,NIC,9918688888,CONSUMPTION-ANY OTHER CONSUMPTION RELATED LOAN...
2506,9999535,CUSTOMER_1426,45,MALE,1.0,LOW,Entrepreneur / Business Owners,MATALE,988,Essential,MARRIED,1980-06-22,SELF-EMPLOYED,976895289567,EIC,9918688888,CONSUMPTION-ANY OTHER CONSUMPTION RELATED LOAN...
2507,9999658,CUSTOMER_2065,29,FEMALE,1.0,LOW,Entrepreneur / Business Owners,GAMPAHA,988,Essential,MARRIED,1996-04-30,SELF-EMPLOYED,744098515H,NIC,9918688888,CONSUMPTION-ANY OTHER CONSUMPTION RELATED LOAN...


# Rule 1

In [7]:
customer_df['Eligibility_Flag'] = 'ELIGIBLE'
customer_df['Rejection_Reason'] = None

In [8]:
customer_df.loc[
    (customer_df['AGE'] < 18) | (customer_df['AGE'] > 80),
    ['Eligibility_Flag', 'Rejection_Reason']
] = ['REJECT', 'Regulatory Age Restriction']


In [9]:
customer_df[customer_df['Eligibility_Flag'] == 'REJECT'][['AGE', 'Eligibility_Flag', 'Rejection_Reason']]


,AGE,Eligibility_Flag,Rejection_Reason
1342,88,REJECT,Regulatory Age Restriction
1394,90,REJECT,Regulatory Age Restriction
1912,90,REJECT,Regulatory Age Restriction
2015,99,REJECT,Regulatory Age Restriction
2124,81,REJECT,Regulatory Age Restriction
2160,82,REJECT,Regulatory Age Restriction
2369,82,REJECT,Regulatory Age Restriction


In [10]:
customer_df['Eligibility_Flag'].value_counts()


ELIGIBLE    2502
REJECT         7
Name: Eligibility_Flag, dtype: int64

In [11]:
eligible_cus_df = customer_df[customer_df['Eligibility_Flag'] == 'ELIGIBLE'].copy()


# Rule 2

In [12]:
active_accounts = account_df[
    (account_df['ACCT_STATUS'] == 'ACTIVE') |
    (account_df['ACCT_CLOSE_DATE'].isna())
]

active_account_count = (
    active_accounts
    .groupby('MASKED_ID')
    .size()
    .reset_index(name='Number_of_Active_Accounts')
)